# SHAPLEY

Lloyd S. Shapley  
A Value for n-person games.  
In contribution to the Theory of Games  
(Annals of mathematical Studies), 1953.

References:  
- https://arxiv.org/pdf/1705.07874
- https://christophm.github.io/interpretable-ml-book/shap.html
- https://www.quantmetry.com/blog/valeurs-de-shapley/  
- https://www.youtube.com/watch?v=tuE875uid1c

# SHAP  (<u>SH</u>apley <u>A</u>dditive ex<u>P</u>lanations)

To help users interpret the predictions of complex models, SHAP assigns each feature an importance value for a particular prediction.  

SHAP methods have a correct computational performance and keep consistency with human intuition.

# [Theory](https://arxiv.org/pdf/1705.07874)

## Introduction

For complex models, such as ensemble methods or deep networks, we cannot use the original model $f$ as its own best explanation because it is not easy to understand.  
Instead, we must use a simpler explanation model $g$, which we define as any interpretable approximation of the original model.  

## Additive Feature Attribution Methods

Let $f$ be the original prediction model to be explained and $g$ the explanation model.  

Here, we focus on local methods designed to explain a prediction $f(x)$ based on a single input $x$, as proposed in LIME.  

Explanation models often use simplified inputs $x'$ that map to the original inputs through a mapping function $x = h_x(x')$.  
(Ex: the mapping $x = h_x(x')$ converts a binary vector of interpretable inputs into the original input space).  

Local methods try to ensure $g(z') \approx f(h_x(z'))$ whenever $z' \approx x'$.  
(Note that $h_x(x') = x$ even though $x'$ may contain less information than $x$ because $h_x$ is specific to the current input $x$.)

**Definition 1 Additive feature attribution methods** have an explanation model that is a linear function of binary variables:  

$$g(z') = \phi_0 + \sum_{i = 1}^{M}\phi_i z_i' \tag{1}$$ 

where $z' \in \{0,1\}^M, M $ is the number of simplified input features, and $\phi_i \in \mathbb{R}$. 

This explanation model, attribute an effect $\phi_i$ to each feature, and summing the effects of all feature attributions approximates the output $f(x)$ of the original model. 

### LIME (<u>L</u>ocal <u>I</u>nterpretable <u>M</u>odel-agnostic <u>E</u>xplanations )

The LIME method interprets individual model predictions based on locally approximating the model around a given prediction.  

To find $\phi$, LIME minimizes the follwing objective function:  

$$\xi = \underset{g \in G}{argmin} \; L(f, g, \pi_{x'}) + \Omega(g) \tag{2}$$  

Where:  

- $L$ the squared loss function: $L(f, g, \pi_{x'}) = \underset{z \in Z}{\sum}[f(h(z)) - g(z')]^2 . \pi_x$

- $f$ the model prediction being explained, $g$ the linear explanation model that LIME tries to fit.

- The local kernel $\pi_{x}$ is the exponential kernel $\pi_{x}(z) = exp(-\frac{D(x,z)^2}{\sigma^2})$ with $D(x,z)$ is a distance function that measures the distance between the original instance $x$ and the perturbed instance $z$.  
Common choices for $D$ include the Euclidean distance or the cosine distance. $\sigma$ is a parameter that controls the width of the kernel, determining how quickly the weights decay with distance.  

- $\Omega(g)$ penalizes the complexity of $g$

The exponential kernel ensures that instances closer to the original instance $x$ have a higher weight, while instances farther away have a lower weight. This helps LIME to approximate the local behavior of the model more accurately.

Since in LIME, $g$ follows the Definition 1 and $L$ is a squared loss, the $\xi$ equation can be solved using penalized linear regression.

Note: run faster than SHAP

!["Local approximation"](./png/LIME.png)

#### The LIME Algorithm:

1. Choose the ML model and a reference point to be explained  

2. Generate points all over the $\mathbb{R^p}$ space  
    (sample $X$ values from a Normal distribution inferred from the training set)  

3. Predict the Y coordinate of the sampled points, using the ML model  
    (the generated points are guaranteed to perfectly lie on the ML surface)  

4. Assign weights based on the closeness to the chosen point  
    (use the LIME Kernel, it assigns higher weights to points closer to the reference)  
    
5. Train Linear Ridge Regression on the generated weighted dataset:  
    $\mathbb{E}[Y] = \beta_0 + \sum \beta_i X_j$. The $\beta$ coefficients are regarded as LIME explanation.

References:
- [towardsdatascience.com](https://towardsdatascience.com/lime-explain-machine-learning-predictions-af8f18189bfe)

### DeepLIFT  

Out of scope

### Layer-Wise Relevance Propagation  

Out of scope

## Classic Shapley Value Estimation 

Source: [arvix.org](https://arxiv.org/pdf/1705.07874)

#### Shapley regression values

Based on the Shapley values:  

$$\phi_i = \underset{S \subseteq N \setminus \{i\}}{\sum}\frac{|S|! \times (|N| - |S| - 1)!}{|N|!}(f(S \cup \{i\}) - f(S)) \tag{3}$$  

The _Shapley regression values_ are feature importances for linear models in the presence of multicollinearity.  

This method requires retraining the model on all feature subsets $S \subseteq N$, where $N$ is the set of all features.  

It assigns an importance value to each feature that represents the effect on the model prediction of including that feature.  

Since the effect of withholding a feature depends on other features in the model, the preceding differences are computed for all possible subsets $S \subseteq N \setminus \{i\}$  

For Shapley regression values, $h_x$ maps $1$ or $0$ to the original input space, where $1$ indicates the input is included in the model, and $0$ indicates exclusion from the model.  

**The Shapley regression values match Definition 1** and are hence an additive feature attribution method.  

#### Shapley sampling values

Shapley sampling values are meant to explain any model by:  

1. Applying sampling approximations to Equation (3), and  

2. Approximating the effect of removing a variable from the model by integrating over samples from the training dataset.   

This eliminates the need to retrain the model and allows fewer than $2^{|N|}$ differences to be computed. Since the explanation model form of Shapley sampling values is the same as that for Shapley regression values, it is also an additive feature attribution method.

#### Quantitative input influence

Quantitative input influence is a broader framework that addresses more than feature attributions. However, as part of its method it independently proposes a sampling approximation to Shapley values that is nearly identical to Shapley sampling values. It is thus another additive feature attribution method.

# Simple Properties

A surprising attribute of the class of additive feature attribution methods is the presence of a single unique solution in this class with three following properties:  

- **Property 1: Local accuracy**

$$f(x) = g(x') = \phi_0 + \sum_{i=1}^{M}\phi_i x_i' \tag{5}$$  

The explanation model $g(x')$ matches the original model $f(x)$ when $x = h_x(x')$.

- **Property 2: Missingness**

$$fx_i' = 0 \Rightarrow \phi_i = 0 \tag{6}$$  

Missingness constrains features where $x_i' = 0$  to have no attributed impact.  
(Note: $x_i'$ is the simplified input which represents feature presence $1$ present, $0$ missing).

- **Property 3: Consistency**

Let $f_x(z') = f(h_x(z'))$ and $z' \setminus i$ denote setting $z_i' = 0$. For any two models $f$ and $f'$, if 

$$f_x'(z') - f_x'(z' \setminus i) \ge f_x(z') - f_x(z' \setminus i) \tag{7}$$  

$\forall z' \in \{0,1\}^M \; \text{ then } \phi_i(f',x) \ge \phi_i(f, x)$.

**Theorem 1**  

Only one possible explanation model $g$ follows Definition 1 and satisfies Properties 1, 2, and 3:  

$$\phi_i(f,x) = \underset{z' \subseteq x'}{\sum}\frac{|z'|! \times (M - |z'| - 1)!}{M!}(f_x(z') - f_x(z' \setminus i)) \tag{8}$$

where $|z'|$ is the number of non-zero entries in $z'$, and $z' \subseteq x'$ represents all $z'$ vectors where the non-zero entries are a subset of the non-zero entries in $x'$.

Under Properties 1-3, for a given simplified input mapping $h_x$, Theorem 1 shows that there is only one possible additive feature attribution method.

# SHAP  (<u>SH</u>apley <u>A</u>dditive ex<u>P</u>lanations) Values

SHAP (SHapley Additive exPlanation) values explain how to get <u>from the base value $\mathbb{E[f(z)]}$</u> that would be predicted if we did not know any features to the current output $f(x)$.

!["SHapley Additive exPlanations"](./png/Additive.png)  
Figure 1

(Note 1: $\phi_0 = \mathbb{E[f(z)]}$)  
(Note 2: When the model is non-linear or the input features are not independent, the order in which features are added to the expectation (or prediction) matters. In this context, SHAP values (SHapley Additive exPlanations) are obtained by averaging the $\phi_i$ values (the marginal contributions of the features) across all possible orders of adding the features.

In other words, SHAP values take into account the effect of each feature on the prediction by considering all possible sequences in which the features can be added. This allows for capturing complex interactions between features and provides a more accurate and fair explanation of the contribution of each feature to the model's prediction.)

Where $f_x(z') = f(h_x(z')) = \mathbb{E}[f(z) | z_S]$, and $S$ is the set of non-zero indexes in $z'$ (Figure 1).

SHAP values provide the unique additive feature importance measure that adheres to Properties 1-3 and uses conditional expectations to define simplified inputs.

Implicit in this definition of SHAP values is a simplified input mapping,  

$h_x(z') = z_S$, where $z_S$ has missing values for features not in the set $S$.  

Since most models cannot handle arbitrary patterns of missing input values, we approximate $f(z_S)$ with $\mathbb{E}[f(z) | z_S]$.

The exact computation of SHAP values is challenging.  
However, by combining insights from current additive feature attribution methods, we can approximate them.

We describe two model-agnostic approximation methods, one that is already known (Shapley sampling values) and another that is novel (Kernel SHAP).

## Model-Agnostic Approximations

Out of scope, see the documentation

Source: [arvix.org](https://arxiv.org/pdf/1705.07874)

## Kernel SHAP (Linear LIME + Shapley values)

Linear LIME uses a linear explanation model to locally approximate $f$, where local is measured in the simplified binary input space.  

Since linear LIME is an additive feature attribution method, we know the Shapley values are the only possible solution to Equation 2 that satisfies Properties 1-3  – local accuracy, missingness and consistency.  

Due to the LIME choices for the loss function $L$ and the local kernel $\pi_x$, , does not recover the Shapley values and as a consequence, local accuracy and/or consistency are violated with LIME.

Below we show how to avoid heuristically choosing the parameters in Equation 2 and how to find the loss function $L$, weighting kernel $\pi_x'$ , and regularization term $\Omega$ that recover the Shapley values.  

**<u>Theorem 2 (Shapley kernel)</u>**  

Under Definition 1, the specific forms of $\pi_x'$ , $L$, and $\Omega$ that make solutions of Equation 2 consistent with Properties 1 through 3 are:  

$$\Omega(g) = 0$$  

$$\pi_{x'}(z') = \frac{(M - 1)}{\binom{M}{|z'|}|z'|(M - |z'|)}$$  

$$L(f,g,\pi_x') = \underset{z' \in Z}{\sum}[f(h_x^{-1}(z')) - g(z')]^2\pi_x'(z')$$  

where $|z'|$ is the number of non-zero elements in $z'$.

It is important to note that $\pi_{x'}(z') = \infty$ when $|z'| \in \{0, M\}$, which enforces $\phi_0 = f_x(\emptyset)$ and $f(x) = \sum_{i=0}^{M}\phi_i$.  

In practice, these infinite weights can be avoided during optimization by analytically eliminating two variables using these constraints. (In our cas we will delete it from $z'$).

Since $g(z')$ in Theorem 2 is assumed to follow a linear form, and $L$ is a squared loss, Equation 2 can still be solved using linear regression. As a consequence, the Shapley values from game theory can be computed using weighted linear regression. 

Estimating all SHAP values using regression provides better sample efficiency than the direct use of classical Shapley equations

## Model-Specific Approximations

### Linear SHAP

**Corollary1 (LinearSHAP)**  

Given a linear model $f(x)=\sum_{j=1}^Mw_j x_j +b : \phi_0(f,x) = b$ and $\phi_j(f,x) = w_j(x_j - \mathbb{E}[x_j])$

### Low-Order SHAP  

### Max SHAP 

### Deep SHAP (DeepLIFT + Shapley values)  

# Computational and User Study Experiments  

See the source

Source: [arvix.org](https://arxiv.org/pdf/1705.07874)

# Conclusion

The growing tension between the accuracy and interpretability of model predictions has motivated the development of methods that help users interpret predictions. The SHAP framework identifies the class of additive feature importance methods (which includes six previous methods) and shows there is a unique solution in this class that adheres to desirable properties. The thread of unity that SHAP weaves through the literature is an encouraging sign that common principles about model interpretation can inform the development of future methods